### Head

In [11]:
from math import log
import copy
import arff

### Calculate Entropy

In [12]:
def calEn(data_train):
    
    total = len(data_train)#the number of the total examples
    
    labels = {} #the label dict,where the value of each label is the total number of it.
    
    for instance in data_train:
        in1 = instance[-1]
        if in1 not in labels.keys():
            labels[in1] = 1
        else:
            labels[in1] += 1     
    
    en = float(0) # the entropy of this dataset
    
    for in2 in labels:
        p = labels[in2]/float(total)
        en -= p * log(p,2)
    return(en)

### Data Split:
1. Normal split:
2. Numerical split:

In [13]:
def dataSplit(dataset, feature , value):
    # in this function, the feature is given by the number in it
    # the first argument is the position, the second one is the value for this point
    newdata = []
    for example in dataset:
            if example[feature] == value:
                temp = example[:feature]
                temp.extend(example[feature+1:])
                newdata.append(temp) #this step will append the newdata
    return(newdata)

def helpNum(numlist):
    #unique first and then do the sort thing
    numlist = list(set(numlist))
    numlist.sort()
    l = len(numlist)
    midpoints = []
    for i in range(1,l):
        midpoints.append( (numlist[i-1]+numlist[i])/2  )
    return(midpoints)

def dataSplitNum(data_train, feature , mid ):
    #this function will return the data splited by the given numerical feature:
    newdataLow = []
    newdataHigh = []
    for example in data_train:
        if example[feature] < mid:
            temp = copy.copy(example)
            newdataLow.append(temp)
        else:
            temp2 = copy.copy(example)
            newdataHigh.append(temp2)
    return(newdataLow,newdataHigh) 

### Find best Mid points/ Feature to split:

In [14]:
def bestNumMid(data_train, label):
    # the total entropy before the split
    all_ent = calEn(data_train)
    values = [example[label] for example in data_train]
    midvalues = helpNum(values)
    #do the entropy calculation:
    infoGain = 0.0
    resultpoint = None
    l = len(midvalues)
    for i in range(l):
        mid = midvalues[i]
        temp_en = 0.0
        new1, new2 = dataSplitNum(data_train, label , mid)
        temp_en += calEn(new1)
        temp_en += calEn(new2)
        temp_infoGain = all_ent - temp_en
        if(temp_infoGain  > infoGain):
            resultpoint = mid
            infoGain = temp_infoGain
    return(resultpoint,infoGain)


def findBestsplit(data_train):
    
    # the total entropy before the split
    all_ent = calEn(data_train)
    # the total number of features
    fe_len = len(data_train[0])-1
    infoGain = float(0)
    
    result_feature = None
    midvaluefornumeric = None
    
    for i in range(fe_len):
        temp_entropy = float(0)
        
        feat = data_train[0][i]
        
        if type(feat) == int or type(feat) == float:
            temp_midvalue,temp_infoGain = bestNumMid(data_train ,i )
            if(temp_infoGain > infoGain):
                infoGain = temp_infoGain
                result_feature = i
                midvaluefornumeric = temp_midvalue           
        else:
            featurelist = []
            for example in data_train:
                if example[i] in featurelist:
                    pass
                else:
                    featurelist.append(example[i])       
            for feature in featurelist:
                kset = dataSplit(data_train , i , feature )
                temp_entropy += calEn(kset)
            temp_infoGain = all_ent - temp_entropy
            if (temp_infoGain > infoGain): # Since we only change the reslut_feature if it's equal and smaller,thus, this will work
                infoGain = temp_infoGain
                result_feature = i
            
    return(result_feature , midvaluefornumeric)


### Make the Node, Make a Tree:

In [32]:
def MakeNode(classes):
    # this fucntion will return a dict which will take the majoity of this class to be the node
    #create a dict to store the node
    number = {}
    for each in classes:
        if each in number:
            number[each] +=1
        else:
            number[each] =1
    #sort the dict by their number:
    new_number = sorted(number.items(), key = lambda item:item[1], reverse = True)
    return(new_number[0][0])

In [19]:
def MakeTreeM(data_train , labels):
    
    classes = [example[-1] for example in data_train]# the calsssse of the dataset
    
    #If data_set all belong to one class   ##or we have fewer than m instances
    if classes.count(classes[0]) == len(classes): #or len(data_train) < m:
        return(classes[0])
    
    #or we will end at the point when there is only one class and we will return the label which has showd a lot
    elif  len(data_train[0]) <= 5:
        return(MakeNode(classes))
    
    else:
        # Add a new node to the end of last chain
        S , M = findBestsplit(data_train)
        print(S)
        
        # what if S is None?
        #if S is None:
         #   return(MakeNode(classes))

        if M is None:
            Node_Name = labels[S]
        else:
            Node_Name = labels[S] + '<' + str(M)
        
        #Create our Tree:
        Tree = {Node_Name:{}}
        
        # the core part where we make a tree:
        if M is None:
            S_values = [example[S] for example in data_train]
            uniqueS_values = set(S_values)
            
            del(labels[S]) # except the S feature we select from above
            for each in uniqueS_values:
                print(each)
                Dnew = dataSplit(data_train , S , each)
                nextlabels = labels[:]
                Tree[Node_Name][each] = MakeTreeM(Dnew , nextlabels)
                
        elif M is not None:
            Dnew = dataSplitNum(data_train , S , M)
            for i in range(2):
                Dnew_temp = Dnew[i]
                newlables = labels[:]
                Tree[Node_Name][i] = MakeTreeM( Dnew_temp , newlables)
            
    return(Tree)

### test sample:

In [20]:
def example():
    data = [['1','1',2,'yes'],
           ['1','1',1,'yes'],
           ['1','0',3,'yes'],
           ['1','0',6,'no'],
           ['0','0',9,'no'],
           ['0','1',5,'no'],
           ['1','0',5,'no']]
    label = ['+','-','one']
    return data,label
x1,x2 = example()
x3 = [2,1,3,6,9,5,5]
print(x1)
print(x2)
print(x3)

[['1', '1', 2, 'yes'], ['1', '1', 1, 'yes'], ['1', '0', 3, 'yes'], ['1', '0', 6, 'no'], ['0', '0', 9, 'no'], ['0', '1', 5, 'no'], ['1', '0', 5, 'no']]
['+', '-', 'one']
[2, 1, 3, 6, 9, 5, 5]


In [21]:
MakeTreeM(x1,x2)

AttributeError: 'dict' object has no attribute 'key'

### Train_data

In [26]:
train_data = arff.load(open('credit_train.arff', 'r'))

In [27]:
data_train = train_data['data']
labels = train_data['attributes']

In [28]:
labels
newlabels = [exampel[0] for exampel in labels]

In [29]:
newlabels = newlabels[0:len(newlabels)-1]
print(newlabels)
labels

['A1', 'A2', 'A3', 'A4', 'A5', 'A8', 'A14', 'A15']


[('A1', ['b', 'a']),
 ('A2', 'REAL'),
 ('A3', 'REAL'),
 ('A4', ['u', 'y', 'l', 't']),
 ('A5', ['g', 'p', 'gg']),
 ('A8', 'REAL'),
 ('A14', 'REAL'),
 ('A15', 'REAL'),
 ('class', ['+', '-'])]

First try:

In [31]:

print(newlabels)

tree1 = MakeTreeM(data_train , newlabels)

['A1', 'A2', 'A3', 'A4', 'A5', 'A8', 'A14', 'A15']
7
5
6
None


TypeError: list indices must be integers or slices, not NoneType

In [30]:
calEn(data_train)

0.8653409924402704